In [1]:
# Import dependencies needes
import pandas as pd
import numpy as np

# File to Load 
file_path = "Resources/vivienda.csv"

# Read Purchasing File and store into Pandas data frame
vivienda = pd.read_csv(file_path, encoding='ISO-8859-1')

C:\Users\joyce\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (12,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
#Get the name of the columns
vivienda.columns

Index(['cve_entidad', 'desc_entidad', 'cve_municipio', 'desc_municipio',
       'id_indicador', 'indicador', '1950', '1960', '1970', '1980', '1990',
       '1995', '2000', '2005', '2010', '2014', '2015', '2020',
       'unidad_medida'],
      dtype='object')

In [3]:
v_columns = vivienda[['cve_entidad','desc_entidad', 'cve_municipio', 'desc_municipio',
       'id_indicador', 'indicador','2020',
       'unidad_medida' ]]

In [4]:
# Check the count of indicators 
i = v_columns['indicador'].value_counts().sort_index()
i

Disponibilidad de bienes: Automóvil o camioneta                               2502
Disponibilidad de bienes: Bicicleta como medio de transporte                  2502
Disponibilidad de bienes: Lavadora                                            2502
Disponibilidad de bienes: Motocicleta o motoneta                              2502
Disponibilidad de bienes: Refrigerador                                        2502
                                                                              ... 
Viviendas que son casa independiente o local no construido para habitación      33
Viviendas que son cuarto de azotea                                              33
Viviendas que son departamento en edificio o vivienda en vecindad               33
Viviendas que son refugio                                                       33
Viviendas, de cada cien que cuentan con electricidad, agua y drenaje            33
Name: indicador, Length: 127, dtype: int64

In [5]:
# Since we saw that there are some indicators that we don't need because of it's value count, we drop them
v_columns2=v_columns[v_columns['indicador'].isin(v_columns['indicador'].value_counts()[v_columns['indicador'].value_counts()>33].index)]

# We verify that the indicators were successfully deleted
v_columns2['indicador'].value_counts().sort_index()

Disponibilidad de bienes: Automóvil o camioneta                                                           2502
Disponibilidad de bienes: Bicicleta como medio de transporte                                              2502
Disponibilidad de bienes: Lavadora                                                                        2502
Disponibilidad de bienes: Motocicleta o motoneta                                                          2502
Disponibilidad de bienes: Refrigerador                                                                    2502
Disponibilidad de servicios en la vivienda: Cisterna o aljibe                                             2502
Disponibilidad de servicios en la vivienda: Tinaco                                                        2502
Porcentaje de viviendas con agua entubada                                                                 2490
Porcentaje de viviendas con agua entubada dentro de la vivienda                                           2502
P

In [6]:
# In this section of code we realized that we have to check the missing values of each 'indicador', so we can drop
# the ones that are not useful because they have excess of NaN's

# Get the list of the values left
i_values = v_columns2['indicador'].unique()

# Initialize list that will save the sum of the missing values
cond=[]

# Initialize list for the boolean array
boolean = []

# Loop to identify the sum of the missing values according to each 'indicador'
for val in i_values:
    df = v_columns2[v_columns2['indicador'] == val]
    cond.append(df['2020'].isna().sum())

# Loop to create a vector with boolean values for the 'indicadores' that have more than one missing value
for x in cond:
    if x>1:
        boolean.append(True)
    else:
        boolean.append(False)
        
# Get the name of the 'indicadores' that have more than one missing value
mv_i = i_values[boolean]

In [7]:
# Initialize our new data frame
v_clean = v_columns2

# Loop to eliminate the 'indicadores' that have too many missing values
for x in mv_i:
    v_clean = v_clean[v_clean['indicador'] != x]
    
# Check the lenght of the 'indicadores' that are left and that do not have NaN's
len(v_clean['indicador'].unique())

20

In [8]:
# Check the lenght of the 'indicadores' before the Nan's were dropped
len(v_columns2['indicador'].unique())

43

In [9]:
# Check the lenght of 'indicadores' that we wanted to drop 
len(mv_i)

23

Since the sum of the  lenght of the 'indicadores' after we cleaned (20) and the lenght of the 'indicadores' that we wanted to drop (23) is 43 we are certain that we dropped the right amount of 'indicadores'

In [10]:
# We have to check the remaining NaN values 
v_clean.isnull().sum()

cve_entidad       0
desc_entidad      0
cve_municipio     0
desc_municipio    0
id_indicador      0
indicador         0
2020              2
unidad_medida     0
dtype: int64

In [11]:
# Check the position of the missing values 
mis_val = v_clean['2020'].isnull()

# Get the name of the two 'indicadores' that have a missing value  
indicador_miss = v_clean['indicador']
indicador_miss[mis_val]

9690            Total de viviendas particulares habitadas
9692    Promedio de ocupantes en viviendas particulare...
Name: indicador, dtype: object

In [12]:
# Get the 'estado' and 'municipio' of the missing values to see if they are relevant
indicador_edo = v_clean['desc_entidad']
indicador_mpo = v_clean['desc_municipio']


pd.DataFrame({'Estado' : indicador_edo[mis_val], 'Municipio' : indicador_mpo[mis_val]})

,Estado,Municipio
9690,Chiapas,Otros municipios
9692,Chiapas,Otros municipios


In [13]:
# Since the data was not relevant we are going to transform NaN to 0 and it will not affect our analysis
v_clean['2020'] = v_clean['2020'].fillna(0)

# Check the count of missing values
v_clean.isnull().sum()

cve_entidad       0
desc_entidad      0
cve_municipio     0
desc_municipio    0
id_indicador      0
indicador         0
2020              0
unidad_medida     0
dtype: int64

In [14]:
# Check the 'indicadores' left
v_clean['indicador'].unique()

array(['Total de viviendas particulares habitadas',
       'Promedio de ocupantes en viviendas particulares habitadas',
       'Porcentaje de viviendas con piso de tierra',
       'Porcentaje de viviendas con electricidad',
       'Porcentaje de viviendas particulares habitadas con disponibilidad de drenaje',
       'Porcentaje de viviendas particulares habitadas con disponibilidad de excusado',
       'Porcentaje de viviendas particulares habitadas que disponen de Internet',
       'Porcentaje de viviendas particulares habitadas que disponen de computadora',
       'Porcentaje de viviendas particulares habitadas que disponen de teléfono fijo',
       'Porcentaje de viviendas particulares habitadas que disponen de teléfono celular',
       'Porcentaje de viviendas particulares habitadas que disponen de televisión de paga',
       'Promedio de ocupantes por cuarto (Índice de hacinamiento)',
       'Porcentaje de viviendas con agua entubada dentro de la vivienda',
       'Disponibilidad 

In [15]:
# List of the 'indicadores' that are not useful
not_useful = ['Total de viviendas particulares habitadas',
 'Promedio de ocupantes en viviendas particulares habitadas',
 'Porcentaje de viviendas con piso de tierra',
 'Promedio de ocupantes por cuarto (Índice de hacinamiento)', 
 'Disponibilidad de bienes: Bicicleta como medio de transporte']

In [16]:
# List of the 'indicadores' that we will use for our analysis
useful = ['Porcentaje de viviendas con electricidad',
 'Porcentaje de viviendas particulares habitadas con disponibilidad de drenaje',
 'Porcentaje de viviendas particulares habitadas con disponibilidad de excusado',
 'Porcentaje de viviendas particulares habitadas que disponen de Internet',
 'Porcentaje de viviendas particulares habitadas que disponen de computadora',
 'Porcentaje de viviendas particulares habitadas que disponen de teléfono fijo',
 'Porcentaje de viviendas particulares habitadas que disponen de teléfono celular',
 'Porcentaje de viviendas particulares habitadas que disponen de televisión de paga',
 'Porcentaje de viviendas con agua entubada dentro de la vivienda',
 'Disponibilidad de servicios en la vivienda: Tinaco',
 'Disponibilidad de servicios en la vivienda: Cisterna o aljibe',
 'Disponibilidad de bienes: Automóvil o camioneta',
 'Disponibilidad de bienes: Motocicleta o motoneta',
 'Disponibilidad de bienes: Refrigerador',
 'Disponibilidad de bienes: Lavadora']

In [17]:
# Initialize our new data frame
v_final = v_clean

# Loop to eliminate the 'indicadores' that we don't need
for u in not_useful:
    v_final = v_final[v_final['indicador'] != u]
    
# Check the lenght of the 'indicadores' that are left (should be the same lenght as the list of useful 'indicadores')
len(v_final['indicador'].unique())

15

In [18]:
# Create an empty list were the values of the index will be added
index = []

# Select the column that contain the data that will be needed to calculate the index values
indicadores = v_final['2020']

# Loop that assings the values 
for ind in indicadores:
    if ind>=0 and ind<10:
        index.append(1)
    elif ind>=10 and ind<20:
        index.append(2)
    elif ind>=20 and ind<30:
        index.append(3)
    elif ind>=30 and ind<40:
        index.append(4)
    elif ind>=40 and ind<50:
        index.append(5)
    elif ind>=50 and ind<60:
        index.append(6)
    elif ind>=60 and ind<70:
        index.append(7)
    elif ind>=70 and ind<80:
        index.append(8)
    elif ind>=80 and ind<90:
        index.append(9)
    else:
        index.append(10)
       
# Verify that the index list is the same length as our final data frame  
len(index)-len(v_final)

0

In [19]:
# Add the Index column to the final data frame
v_final['Index'] = index

# Verify that the Index column was added 
v_final.head(5)

,cve_entidad,desc_entidad,cve_municipio,desc_municipio,id_indicador,indicador,2020,unidad_medida,Index
12,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,3114006001,Porcentaje de viviendas con electricidad,99.001843,Porcentaje,10
13,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,6200031290,Porcentaje de viviendas particulares habitadas...,95.469330,Porcentaje,10
14,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,6200031291,Porcentaje de viviendas particulares habitadas...,98.070381,Porcentaje,10
34,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,6207019042,Porcentaje de viviendas particulares habitadas...,52.072835,Porcentaje,6
37,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,6207019046,Porcentaje de viviendas particulares habitadas...,37.559287,Porcentaje,4


In [20]:
# In this section we calculate the value of the index created to measure the economic status of the country 

# Filter the data base to get the values for the whole country
country = v_final[v_final['desc_entidad'] == 'Estados Unidos Mexicanos']

# Calculate the index value
country_index = country['Index'].sum()

# Create a dictionary to convert it to Data Frame
country_dic = {'Country' :['Mexico'], 'Index_Value' : [country_index]}

# Convert to Data Frame
country_df = pd.DataFrame(country_dic)

# Show the result
country_df


,Country,Index_Value
0,Mexico,100


In [26]:
# In this section we calculate the value of the index created to measure the economic status for each state of the country

# Create a list with the names of the states
st = v_final['desc_entidad'].unique()
state = st.tolist()
state.remove('Estados Unidos Mexicanos')

# Create a list where the index values will be saved
st_index = []

# Loop to calculate the index value for each state
for s in state:
    df = v_final[v_final['desc_entidad'] == s]
    df_edo = df[df['desc_municipio'] == 'Estatal']
    st_index.append(df_edo['Index'].sum())
    
# Create a dictionary to convert it to Data Frame
state_dic = {'State' : state, 'Index_Value' : st_index}

# Convert to Data Frame
state_df = pd.DataFrame(state_dic)

# Show the result
state_df

,State,Index_Value
0,Aguascalientes,111
1,Baja California,105
2,Baja California Sur,108
3,Campeche,99
4,Coahuila de Zaragoza,107
5,Colima,107
6,Chiapas,81
7,Chihuahua,103
8,Ciudad de México,117
9,Durango,101


In [28]:
state_df.to_csv("../Output/PODERADQUISITIVOPORESTADO.csv", index=False, header=True)

In [93]:
# In this section we calculate the value of the index created to measure the economic status for each municipality
# of the country

# Create a list with the names of the states
st = v_final['desc_entidad'].unique()
state = st.tolist()
state.remove('Estados Unidos Mexicanos')

# Create a list where the index values will be saved
mun_index = []

# Create a list where the name of the state will be saved
st_list = []

# Create a list where the name of the municipality will be saved
mun_list = []

# Loop to calculate the index value for each municipality
for s in state:
    df = v_final[v_final['desc_entidad'] == s]
    mun = df['desc_municipio'].unique()
    mun = mun.tolist()
    mun.remove('Estatal')
    
    for m in mun:
        st_list.append(s)
        df_mun = df[df['desc_municipio'] == m]
        mun_index.append(df_mun['Index'].sum())
        mun_list.append(m)
    
# Create a dictionary to convert it to Data Frame
mun_dic = {'State' : st_list, 'Municipality' : mun_list, 'Index_Value' : mun_index}

# Convert to Data Frame
mun_df = pd.DataFrame(mun_dic)

# Show the result
mun_df


,State,Municipality,Index_Value
0,Aguascalientes,Aguascalientes,113
1,Aguascalientes,Asientos,98
2,Aguascalientes,Calvillo,104
3,Aguascalientes,Cosío,98
4,Aguascalientes,Jesús María,112
...,...,...,...
2464,Zacatecas,Villa Hidalgo,92
2465,Zacatecas,Villanueva,105
2466,Zacatecas,Zacatecas,115
2467,Zacatecas,Trancoso,94


In [ ]:
# Export file as a CSV, without the Pandas index, but with the header
mun_df.to_csv("../Output/PODERADQUISITIVOPORESTADOMUNICIPIO.csv", index=False, header=True)